# Introdução à Feature Store com Databricks Unity Catalog

A Feature Store é um repositório centralizado de features para utilização em treinamento e inferência de modelos de machine learning, garantindo consistência e escalabilidade. Os principais benefícios de utilizá-la são:

- Compartilhamento de features.
- Reutilização e rastreabilidade aprimoradas.
- Consistência no cálculo de features para treinamento e inferência.

## Conteúdo deste Notebook

Este notebook cobre as principais etapas para utilização da Feature Store para treinamento de modelos de machine learning.

- Ingestão de dados e salvamento como uma feature table no Unity Catalog.
- Carregamento de Features.
- Criação de dataset de treinamento a partir de uma Feature Store.
- Treinamento de um modelo simples


# Instalar bibliotecas

In [0]:
%pip install databricks-feature-engineering==0.7.0 databricks-sdk==0.20.0
dbutils.library.restartPython()

In [0]:
from databricks.feature_engineering import FeatureEngineeringClient, FeatureLookup

# Carregar dados

In [0]:
query = f"""
    SELECT 
    sha2(concat(dropoff_zip, '-', pickup_zip, '-', tpep_pickup_datetime), 256) as trip_sk
    , dropoff_zip
    , pickup_zip
    , date_part('month', tpep_pickup_datetime) as trip_month
    , date_part('day', tpep_pickup_datetime) as trip_day
    , date_part('dayofweek', tpep_pickup_datetime) as trip_dow
    , date_part('hour', tpep_pickup_datetime) as trip_pickup_hour
    , datediff(minute, tpep_pickup_datetime, tpep_dropoff_datetime) as trip_duration
    , trip_distance
    , fare_amount
    FROM samples.nyctaxi.trips
"""

ft_nyc_taxi_trips = spark.sql(query)

In [0]:
# Podemos utilizar a Pandas API para visualizar os resultados
df = ft_nyc_taxi_trips.pandas_api()
display(df)

In [0]:
# Para evitar o vazamendo de dados realiza-se o drop da variável alvo
df = df.drop("fare_amount", axis=1)
df.head(3)

### Salvar a tabela de features
Em seguida, vamos salvar nossa tabela de features como uma Tabela de Engenharia de Features utilizando o método `create_table`.

Será necessário atribuir um nome à tabela e definir uma chave primária que será usada para consultas. A chave primária deve ser única.

In [0]:
fe = FeatureEngineeringClient()
fe.create_table(
    name="<catalog>.<schema>.fs_nyc_taxi_trips",
    primary_keys=["trip_sk"],
    df=df.to_spark(),
    description="Taxi trips features",
)

# Acessar metadados

In [0]:
fe_get_table = fe.get_table(name="<catalog>.<schema>.fs_nyc_taxi_trips")
print("A feature store contém as features: ", fe_get_table.features)


## Treinando um modelo a partir da Feature Store 

### Construção do Dataset de Treinamento

Iniciamos a construção do dataset de treinamento selecionando os IDs e a variável-alvo diretamente da tabela de origem. 

#### Etapas:

1. Selecionamos as colunas relevantes:
   - **`trip_sk`**: O identificador único para cada registro (*primary key*).
   - **`fare_amount`**: A variável-alvo que será prevista pelo modelo.



In [0]:
spine_table = ft_nyc_taxi_trips.select("trip_sk", "fare_amount")
display(spine_table)

### Criando o Conjunto de Treinamento com Feature Lookup

Agora, utilizando os IDs, podemos resgatar as features previamente criadas e disponibilizadas na Feature Store através de **Feature Lookup**. Em seguida, criamos o conjunto de treinamento utilizando o método `create_training_set` do **FeatureEngineeringClient**.

#### Etapas:

1. Definimos os *lookups* para as features que queremos usar no treinamento. 
   - Utilizamos o nome da tabela na Feature Store (`table_name`) e a chave de correspondência (`lookup_key`) que será utilizada para unir os dados originais com as features.
   
2. Criamos o conjunto de treinamento (*training set*) unindo o DataFrame original (`spine_table`) com os dados da Feature Store. Também definimos a variável-alvo (label) que será usada no modelo.

In [0]:
model_feature_lookups = [
      FeatureLookup(
          table_name="<catalog>.<schema>.fs_nyc_taxi_trips",
          lookup_key=["trip_sk"],
      )
]

training_set = fe.create_training_set(
    df=spine_table, 
    feature_lookups=model_feature_lookups,
    label='fare_amount',
)

# Convert to Pandas
training_pd = training_set.load_df().toPandas()
training_pd.head(2)

### Dividir o Dataset

In [0]:
from sklearn.model_selection import train_test_split
target="fare_amount"
training_pd = training_pd.set_index("trip_sk")
X_train, X_test, y_train, y_test = train_test_split(training_pd.drop(target, axis=1), training_pd[target], test_size=.2)

### Treinar o modelo

In [0]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Train the model
xgb_model = xgb.XGBRegressor(enable_categorical=True)
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
              
# Print the metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")
